In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Large_B-cell_Lymphoma"
cohort = "GSE114022"

# Input paths
in_trait_dir = "../../input/GEO/Large_B-cell_Lymphoma"
in_cohort_dir = "../../input/GEO/Large_B-cell_Lymphoma/GSE114022"

# Output paths
out_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/GSE114022.csv"
out_gene_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE114022.csv"
out_clinical_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/clinical_data/GSE114022.csv"
json_path = "../../output/preprocess/Large_B-cell_Lymphoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The  ETS  inhibitors  YK-4-279  and  TK-216  have  anti-tumor  activity  in  lymphomas  and  interfere  with  SPIB  in  activated  B  cell-like  type  diffuse  large  B  cell  lymphoma"
!Series_summary	"TMD8 and U2932 were treated with YK-4-279 for 4 and 8 hours."
!Series_overall_design	"Identifing genes modulated by the YK-4-279 in human lymphoma cell lines"
Sample Characteristics Dictionary:
{0: ['cell line: TMD8', 'cell line: U2932'], 1: ['treatment: YK-S', 'treatment: YK-R', 'treatment: DMSO'], 2: ['time point: 4hr', 'time point: 8hr']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data
# as it mentions "Identifing genes modulated by the YK-4-279 in human lymphoma cell lines"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Large B-cell Lymphoma):
# From the characteristics dictionary, we can see this is a cell line experiment
# Cell lines (TMD8 and U2932) are mentioned as lymphoma cell lines in the Series_title
# Cell lines can be used as a proxy for the trait, as they represent the disease
trait_row = 0  # The row containing cell line information

# No age information is available in this cell line experiment
age_row = None

# No gender information is available in this cell line experiment
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    """
    Convert the cell line information to a binary trait variable.
    Both cell lines represent lymphoma, so we'll use 1 for all samples.
    """
    if not isinstance(value, str):
        return None
    # Since both cell lines represent lymphoma (the trait of interest), we'll map both to 1
    # Extract the value after the colon
    if ':' in value:
        cell_line = value.split(':', 1)[1].strip()
        if cell_line in ['TMD8', 'U2932']:
            return 1
    return None

def convert_age(value):
    """Placeholder function for age conversion"""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion"""
    return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Initial dataframe set in the previous step would be called clinical_data
    # Let's assume it's available
    if 'clinical_data' in locals() or 'clinical_data' in globals():
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the selected clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:", preview)
        
        # Save the clinical data to a CSV file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=True)
        print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features: {'GSM3130825': [1.0], 'GSM3130826': [1.0], 'GSM3130827': [1.0], 'GSM3130828': [1.0], 'GSM3130829': [1.0], 'GSM3130830': [1.0], 'GSM3130831': [1.0], 'GSM3130832': [1.0], 'GSM3130833': [1.0], 'GSM3130834': [1.0], 'GSM3130835': [1.0], 'GSM3130836': [1.0], 'GSM3130837': [1.0], 'GSM3130838': [1.0], 'GSM3130839': [1.0], 'GSM3130840': [1.0], 'GSM3130841': [1.0], 'GSM3130842': [1.0], 'GSM3130843': [1.0], 'GSM3130844': [1.0], 'GSM3130845': [1.0], 'GSM3130846': [1.0], 'GSM3130847': [1.0], 'GSM3130848': [1.0], 'GSM3130849': [1.0], 'GSM3130850': [1.0], 'GSM3130851': [1.0], 'GSM3130852': [1.0], 'GSM3130853': [1.0], 'GSM3130854': [1.0]}
Clinical data saved to ../../output/preprocess/Large_B-cell_Lymphoma/clinical_data/GSE114022.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"The  ETS  inhibitors  YK-4-279  and  TK-216  have  anti-tumor  activity  in  lymphomas  and  interfere  with  SPIB  in  activated  B  cell-like  type  diffuse  large  B  cell  lymphoma"
Line 1: !Series_geo_accession	"GSE114022"
Line 2: !Series_status	"Public on May 01 2021"
Line 3: !Series_submission_date	"May 03 2018"
Line 4: !Series_last_update_date	"May 02 2021"
Line 5: !Series_summary	"TMD8 and U2932 were treated with YK-4-279 for 4 and 8 hours."
Line 6: !Series_overall_design	"Identifing genes modulated by the YK-4-279 in human lymphoma cell lines"
Line 7: !Series_type	"Expression profiling by array"
Line 8: !Series_contributor	"Luciano,,Cascione"
Line 9: !Series_contributor	"Francesco,,Bertoni"
Found table marker at line 61
First few lines after marker:
"ID_REF"	"GSM3130825"	"GSM3130826"	"GSM3130827"	"GSM3130828"	"GSM3130829"	"GSM3130830"	"GSM3130831"	"GSM3130832"	"GSM3130833"	"GSM3130834"	"GSM3130835"	"GSM3130836"	"GSM313

### Step 4: Gene Identifier Review

In [5]:
# The identifiers in the gene expression data start with "ILMN_" which indicates they are Illumina probe IDs
# Illumina probe IDs are not human gene symbols but rather platform-specific identifiers
# They need to be mapped to standard gene symbols for meaningful analysis

# These "ILMN_" identifiers are from Illumina microarray platforms and require mapping to 
# standard gene symbols for interpretation and cross-platform compatibility

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1465067 rows

Gene annotation preview (first few rows):
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate columns for mapping
# From previous output, we can see:
# - Gene expression data uses Illumina probe IDs (ILMN_*) in the 'ID' column
# - Gene annotation has 'ID' column for probe IDs and 'Symbol' column for gene symbols

print("Creating gene mapping...")
# Extract the mapping between probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Successfully created mapping with {len(mapping_df)} entries")

# Preview the mapping to verify structure
print("\nMapping preview (first 5 rows):")
print(preview_df(mapping_df))

# 2. Check the mapping coverage of the gene expression data
probes_in_gene_data = set(gene_data.index)
probes_in_mapping = set(mapping_df['ID'])
common_probes = probes_in_gene_data.intersection(probes_in_mapping)

print(f"\nGene expression data contains {len(probes_in_gene_data)} unique probe IDs")
print(f"Mapping data contains {len(probes_in_mapping)} unique probe IDs")
print(f"Number of probes that can be mapped: {len(common_probes)} ({len(common_probes)/len(probes_in_gene_data)*100:.2f}%)")

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
print("\nApplying gene mapping to convert probe measurements to gene expression...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Successfully created gene expression data with {len(gene_data)} genes")

# Preview the gene expression data
print("\nGene expression data preview (first few genes):")
print(preview_df(gene_data))

# Save the gene expression data to a file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Creating gene mapping...
Successfully created mapping with 44837 entries

Mapping preview (first 5 rows):
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}

Gene expression data contains 47231 unique probe IDs
Mapping data contains 44837 unique probe IDs
Number of probes that can be mapped: 43961 (93.08%)

Applying gene mapping to convert probe measurements to gene expression...
Successfully created gene expression data with 21372 genes

Gene expression data preview (first few genes):
{'GSM3130825': [10.42, 14.17, 13.600000000000001, 18.33, 9.54], 'GSM3130826': [10.59, 13.469999999999999, 13.6, 18.43, 9.26], 'GSM3130827': [10.14, 14.49, 13.57, 18.17, 9.41], 'GSM3130828': [10.42, 14.09, 13.61, 18.24, 9.47], 'GSM3130829': [9.879999999999999, 14.0, 13.5, 18.560000000000002, 9.27], 'GSM3130830': [9.53, 13.64, 14.47, 18.25, 9.18], 'GSM3


Gene expression data saved to ../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE114022.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Gene data was already normalized and saved in the previous step
print("\nChecking normalized gene data availability...")
try:
    # Load the already normalized gene data from the correct path
    if os.path.exists(out_gene_data_file):
        print(f"Loading normalized gene data from {out_gene_data_file}...")
        gene_data = pd.read_csv(out_gene_data_file, index_col=0)
        print(f"Loaded gene data with shape: {gene_data.shape}")
        print(f"Sample of gene symbols: {gene_data.index[:5].tolist()}")
        is_gene_available = True
    else:
        print(f"ERROR: Normalized gene data file not found at {out_gene_data_file}")
        is_gene_available = False
except Exception as e:
    print(f"Error loading normalized gene data: {str(e)}")
    is_gene_available = False

# 2. Check clinical data availability
print("\nChecking clinical data availability...")
# From previous steps, we determined that trait_row was None
trait_row = None
is_trait_available = trait_row is not None

if is_trait_available:
    print("Clinical data is available for this cohort.")
    # This block won't execute since trait_row is None
else:
    print("No clinical data available for this cohort. Cannot proceed with linking.")

# 3. Since clinical data is not available, we can't create proper linked data
# But we'll use the gene data for diagnostic purposes
linked_data = gene_data.T if is_gene_available else pd.DataFrame()
print(f"Created diagnostic data frame with shape: {linked_data.shape}")

# 4. Since we can't evaluate trait bias without trait data, mark as biased
is_biased = True

# 5. Validate and save cohort information
print("\nPerforming final validation...")
note = "Dataset contains gene expression data but lacks clinical information for Large_B-cell_Lymphoma. " \
       "The samples appear to be B-cells at different stages of differentiation, not lymphoma cases."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Report final status - we don't save the linked data because it's not usable
print(f"\nDataset usability for {trait} association studies: {is_usable}")
if not is_usable:
    print(f"Reason: {note}")
else:
    # This block won't execute since is_usable should be False
    print(f"Saving linked data to {out_data_file}")
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)


Checking normalized gene data availability...
Loading normalized gene data from ../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE114022.csv...
Loaded gene data with shape: (21372, 30)
Sample of gene symbols: ['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1']

Checking clinical data availability...
No clinical data available for this cohort. Cannot proceed with linking.
Created diagnostic data frame with shape: (30, 21372)

Performing final validation...
A new JSON file was created at: ../../output/preprocess/Large_B-cell_Lymphoma/cohort_info.json

Dataset usability for Large_B-cell_Lymphoma association studies: False
Reason: Dataset contains gene expression data but lacks clinical information for Large_B-cell_Lymphoma. The samples appear to be B-cells at different stages of differentiation, not lymphoma cases.
